# General practitioner referral

This document shows an example of evolution of a prompt from a simple text to a full prompt with a workflow. It uses Semantic Kernel.

Goal: Improve the referral workflow of a GP practice, to automate documentation generation and delivery.
Scenario: A Dr. Smith sees a patient, 80 year old man, Fred, at his practice. Fred has broken his upper elbow by falling off his ebike. He wants to refer Fred to the Dr. Jones, at the Fracture Clinic in Contoso Hospital.

Inputs:
1. Medical short hand Dr. Smith creating during the visit:

```text
S: Pt c/o pain in R elbow after fall. Hx of HTN, T2DM. O: R elbow swollen, tender to palpation. ROM limited. No obvious deformity. A: Suspected R upper elbow fx. P: X-ray ordered. RICE, analgesics, sling for immobilization. Ref to Fracture Clinic.
```

1. Transcription from the visit.

```text
I saw Fred today. He tells me he fell off his ebike and landed heavily on his elbow. I've sent him to get X-rays and will refer him to Fracture Clinic at Contoso Health.
```

In [180]:
#r "nuget: Microsoft.SemanticKernel, 0.12.207.1-preview"
#r "nuget: Newtonsoft.Json"

#!import config/Settings.cs

using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.SemanticFunctions;
using Microsoft.SemanticKernel.AI.ChatCompletion;
using Newtonsoft.Json;

IKernel kernel = KernelBuilder.Create();

// Configure AI backend used by the kernel
var (useAzureOpenAI, model, azureEndpoint, apiKey, orgId) = Settings.LoadFromFile();
if (useAzureOpenAI) {
    kernel.Config.AddAzureTextCompletionService("davinci", model, azureEndpoint, apiKey);
    kernel.Config.AddAzureChatCompletionService("chatgpt", "gpt-35-turbo", azureEndpoint, apiKey);
}
else
    kernel.Config.AddOpenAITextCompletionService("davinci", model, apiKey, orgId);
    kernel.Config.AddOpenAIChatCompletionService("chatgpt", "gpt-35-turbo", apiKey, orgId);

Installed Packages Microsoft.SemanticKernel, 0.12.207.1-preview Newtonsoft.Json, 13.0.3

Error: (143,26): error CS0104: 'JsonSerializer' is an ambiguous reference between 'Newtonsoft.Json.JsonSerializer' and 'System.Text.Json.JsonSerializer'
(231,47): error CS0104: 'JsonSerializer' is an ambiguous reference between 'Newtonsoft.Json.JsonSerializer' and 'System.Text.Json.JsonSerializer'

Let's create a semantic function used to explain medical shorthand:

In [181]:
string explainMedicalShorthand = """
Explain the following medical shorthand and what it means.

### Medical shorthand
{{$input}}
""";

Let's configure the prompt, e.g. allowing for some creativity and a sufficient number of tokens.

In [182]:
var promptConfig = new PromptTemplateConfig
{
    Completion =
    {
        MaxTokens = 2000,
        Temperature = 0.0,
        TopP = 0.5,
    }
};

The following code prepares an instance of the template, passing in the TXT and configuration above, 
and a couple of other parameters (how to render the TXT and how the template can access other functions).

In [183]:
var explainMedicalShorthandTemplate = new PromptTemplate(
    explainMedicalShorthand,       // Prompt template defined in natural language
    promptConfig,                  // Prompt configuration
    kernel                         // SK instance
);

Let's transform the prompt template into a function that the kernel can execute:

In [184]:
var explainMedicalShorthandFunctionConfig = new SemanticFunctionConfig(promptConfig, explainMedicalShorthandTemplate);

var explainMedicalShorthandFunction = kernel.RegisterSemanticFunction("ExplainMedicalShorthand", "ExplainMedicalShorthand", explainMedicalShorthandFunctionConfig);

Set up some medical shorthand to explain. The medical shorthand is from a patient visit from a upper elbow fracture event.

In [185]:
var medicalShorthand = """
S: Pt c/o pain in R elbow after fall. Hx of HTN, T2DM. O: R elbow swollen, tender to palpation. ROM limited. No obvious deformity. A: Suspected R upper elbow fx. P: X-ray ordered. RICE, analgesics, sling for immobilization. Ref to Fracture Clinic.
""";

...and run the summary function:

In [186]:
var explainedMedicalShorthand = await kernel.RunAsync(medicalShorthand, explainMedicalShorthandFunction);

Console.WriteLine(explainedMedicalShorthand);



S: Patient complains of pain in right elbow after fall. History of hypertension and type 2 diabetes.

O: Right elbow is swollen and tender to palpation. Range of motion is limited. No obvious deformity.

A: Suspected right upper elbow fracture.

P: X-ray ordered. Rest, Ice, Compression, Elevation, analgesics, and sling for immobilization. Referral to Fracture Clinic.


Create a semantic function used to explain medical shorthand and output it as a JSON string.

In [187]:
string explainMedicalShorthandToJson = """
You are a medical AI assistant for applications at a general medical practice. You help produce professional documentation and make recommendations on documentation gaps.

###
Prompt: Explain the meaning of the medical shorthand and output as JSON.
### Medical Shorthand
S: 70 y/o pt c/o CP. O: Pt appears uncomfortable, clutching chest. Vitals: BP 160/90, HR 110, RR 20, O2 sat 95% on RA. EKG shows ST elevations in leads II, III, and aVF. A: Suspected acute MI. P: Administer aspirin, nitroglycerin, and morphine. Transfer to ED for further management.
Response:
{
    "Subjective": "70 year old patient complains of chest pain.",
    "Objective": "Patient appears uncomfortable, clutching chest. Vital signs: Blood pressure 160/90, heart rate 110, respiratory rate 20, oxygen saturation 95% on room air. Electrocardiogram shows ST elevations in leads II, III, and aVF.",
    "Assessment": "Suspected acute myocardial infarction.",
    "Plan": [
        "Administer aspirin, nitroglycerin, and morphine."
        "Transfer to emergency department for further management."
    ]
}

###
Prompt: Explain the meaning of the medical shorthand and output as JSON.
### Medical shorthand
{{$input}}
Response:
""";

Create a new Semantic Function that will be used to explain the medical shorthand as JSON.

In [188]:
var explainMedicalShorthandToJsonTemplate = new PromptTemplate(
    explainMedicalShorthandToJson,       // Prompt template defined in natural language
    promptConfig,                        // Prompt configuration
    kernel                               // SK instance
);

var explainMedicalShorthandAsJsonFunctionConfig = new SemanticFunctionConfig(promptConfig, explainMedicalShorthandToJsonTemplate);

var explainMedicalShorthandAsJsonFunction = kernel.RegisterSemanticFunction("ExplainMedicalShorthandAsJson", "ExplainMedicalShorthandAsJson", explainMedicalShorthandAsJsonFunctionConfig);

...and run the summary function:

In [189]:
var explainedMedicalShorthandAsJson = await kernel.RunAsync(medicalShorthand, explainMedicalShorthandAsJsonFunction);

Console.WriteLine(explainedMedicalShorthandAsJson);


{
    "Subjective": "Patient complains of pain in right elbow after fall. History of hypertension and type 2 diabetes.",
    "Objective": "Right elbow is swollen and tender to palpation. Range of motion is limited. No obvious deformity.",
    "Assessment": "Suspected right upper elbow fracture.",
    "Plan": [
        "Order X-ray.",
        "Rest, ice, compression, and elevation. Analgesics and sling for immobilization.",
        "Refer to fracture clinic."
    ]
}


In [190]:
// Convert String to JSON
class soap {
  public string Subjective { get; set; }
  public string Objective { get; set; }
  public string Assessment { get; set; }
  public List<string> Plan { get; set; }
}

var expandedMedicalShorthandSoap = JsonConvert.DeserializeObject<soap>(explainedMedicalShorthandAsJson.Result);
Console.WriteLine($"Subjective: {expandedMedicalShorthandSoap.Subjective}");
Console.WriteLine($"Objective: {expandedMedicalShorthandSoap.Objective}");
Console.WriteLine($"Assessment: {expandedMedicalShorthandSoap.Assessment}");
foreach (var plan in expandedMedicalShorthandSoap.Plan)
{
    Console.WriteLine($"Plan: {plan}");
}


Subjective: Patient complains of pain in right elbow after fall. History of hypertension and type 2 diabetes.
Objective: Right elbow is swollen and tender to palpation. Range of motion is limited. No obvious deformity.
Assessment: Suspected right upper elbow fracture.
Plan: Order X-ray.
Plan: Rest, ice, compression, and elevation. Analgesics and sling for immobilization.
Plan: Refer to fracture clinic.
